In [49]:
import datetime,time
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate,GridSearchCV
from lightgbm import LGBMRegressor,LGBMClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder,PolynomialFeatures
le=LabelEncoder()
import optuna
from sklearn.preprocessing import SplineTransformer
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")

In [50]:

df=pd.read_excel('mojworld_biomass_weight_dataset.xlsx')
df.to_csv('mojworld_biomass_weight_dataset.csv',index=False)

In [51]:
df

,"ID(block,s/n)",F_time(A),Nano_P_conc(B),Carbon:Nitrogen(C),Biomass_wght(R)
0,"1,s/n1",0,0,0,8.6
1,"1,s/n2",1,1,1,9.4
2,"1,s/n3",2,2,2,19.4
3,"1,s/n4",1,2,0,12.4
4,"1,s/n5",2,1,0,9.7
5,"1,s/n6",1,0,2,14.9
6,"1,s/n7",2,0,1,14.3
7,"1,s/n8",0,1,2,12.9
8,"1,s/n9",0,2,1,8.6
9,"2,s/n1",1,0,0,13.7


In [52]:
top20_corrs = abs(df.corr()['Biomass_wght(R)']).sort_values(ascending = False).head(30)
top20_corrs

Biomass_wght(R)       1.000000
F_time(A)             0.583912
Carbon:Nitrogen(C)    0.365078
Nano_P_conc(B)        0.010597
Name: Biomass_wght(R), dtype: float64

In [53]:
col=df.drop(['ID(block,s/n)','Biomass_wght(R)',],axis=1).columns

In [54]:
from sklearn.model_selection  import train_test_split
X=df[col]
y = df['Biomass_wght(R)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
def evaluate_models(X, y):
    results = []
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    models = [
                #('ctb' , CatBoostRegressor())
#               ( 'xgr' ,XGBRegressor())

    for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append((name, mse,mae,r2))

    df = pd.DataFrame(results, columns=['Model', 'mse', 'mae', 'r2'])
    df_styled = df.style.highlight_max(subset=['mse', 'mae', 'r2'], color='lightblue')
    display(df_styled)

In [31]:
evaluate_models(X,y)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.


LightGBMError: Do not support special JSON characters in feature name.